In [55]:
!pip install kaggle

In [56]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [57]:
!kaggle datasets download -d misrakahmed/vegetable-image-dataset


Dataset URL: https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset
License(s): CC-BY-SA-4.0
vegetable-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [58]:
import zipfile

dataset_zip = "vegetable-image-dataset.zip"  # Change si besoin
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall("dataset/")  # Dossier de destination

print("✅ Dataset téléchargé et extrait avec succès !")


✅ Dataset téléchargé et extrait avec succès !


In [ ]:
!apt-get install -y libimage-exiftool-perl


In [ ]:
!pip install tensorflow
!pip install opencv-python-headless

In [44]:
import os
import cv2
import numpy as np

def remove_blurry_images_from_all_folders(base_folder, threshold=100):
    blurry_images = []

    # Parcours tous les sous-dossiers (train, test, validation)
    for folder in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder)

        if os.path.isdir(folder_path):  # Vérifier si c'est un dossier
            # Parcours tous les sous-dossiers de légumes (par exemple: 'Radish', 'Cucumber', etc.)
            for vegetable_folder in os.listdir(folder_path):
                vegetable_folder_path = os.path.join(folder_path, vegetable_folder)

                if os.path.isdir(vegetable_folder_path):  # Vérifier si c'est un sous-dossier de légumes
                    # Parcours toutes les images dans ce sous-dossier
                    for filename in os.listdir(vegetable_folder_path):
                        img_path = os.path.join(vegetable_folder_path, filename)

                        # Vérifie si c'est une image valide (extensions comme .jpg, .png, etc.)
                        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                            continue

                        img = cv2.imread(img_path)

                        # Vérifie si l'image a été correctement chargée
                        if img is None:
                            print(f"Warning: Could not load image {img_path}")
                            continue

                        # Conversion en niveaux de gris et calcul de la variance de Laplacian
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        variance = cv2.Laplacian(gray, cv2.CV_64F).var()

                        # Si la variance est inférieure au seuil, l'image est floue
                        if variance < threshold:
                            blurry_images.append(img_path)

    return blurry_images

# Exemple d'utilisation avec le dossier de base
blurry_images = remove_blurry_images_from_all_folders('/content/dataset/Vegetable Images')
print("Images floues:", blurry_images)


Images floues: ['/content/dataset/Vegetable Images/train/Bottle_Gourd/0285.jpg', '/content/dataset/Vegetable Images/train/Carrot/0032.jpg', '/content/dataset/Vegetable Images/train/Carrot/0073.jpg', '/content/dataset/Vegetable Images/train/Carrot/0295.jpg', '/content/dataset/Vegetable Images/train/Carrot/0598.jpg', '/content/dataset/Vegetable Images/train/Carrot/0604.jpg', '/content/dataset/Vegetable Images/train/Carrot/0188.jpg', '/content/dataset/Vegetable Images/train/Carrot/0106.jpg', '/content/dataset/Vegetable Images/train/Carrot/0293.jpg', '/content/dataset/Vegetable Images/train/Carrot/0069.jpg', '/content/dataset/Vegetable Images/train/Carrot/0963.jpg', '/content/dataset/Vegetable Images/train/Carrot/0851.jpg', '/content/dataset/Vegetable Images/train/Carrot/0002.jpg', '/content/dataset/Vegetable Images/train/Carrot/0389.jpg', '/content/dataset/Vegetable Images/train/Carrot/0700.jpg', '/content/dataset/Vegetable Images/train/Carrot/0404.jpg', '/content/dataset/Vegetable Images

In [45]:
from PIL import Image
import os

def check_resolution(image_path, min_width=200, min_height=200):
    """Vérifie si l'image a une résolution suffisante."""
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width >= min_width and height >= min_height
    except Exception as e:
        print(f"Erreur lors de l'ouverture de l'image {image_path}: {e}")
        return False  # Considère l'image comme invalide en cas d'erreur

for img_path in blurry_images:  # Remplacement de 'image_paths' par 'blurry_images'
    if not check_resolution(img_path):
        os.remove(img_path)  # Supprime les images qui ne respectent pas les dimensions minimales
        print(f"Supprimé : {img_path}")


In [46]:
!pip install opencv-python numpy


In [47]:
#Correction de la Balance des Blancs

In [48]:
import cv2
import numpy as np
import os

def white_balance(img):
    """Correction de la balance des blancs avec l'espace couleur LAB"""
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.mean(result[:, :, 1])
    avg_b = np.mean(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)

input_folder = "dataset"
output_folder = "enhanced_dataset"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

files_processed = 0  # Compteur d'images traitées

# Parcourir tous les sous-dossiers et fichiers dans le dossier 'dataset'
for root, dirs, files in os.walk(input_folder):
    for file_name in files:
        img_path = os.path.join(root, file_name)

        # Vérifier si c'est un fichier image
        if not file_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            print(f" Fichier ignoré (pas une image) : {file_name}")
            continue

        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠️ Impossible de charger : {img_path}")
        else:
            wb_img = white_balance(img)

            # Construire le chemin de sortie en conservant la structure du dossier
            relative_path = os.path.relpath(root, input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)

            if not os.path.exists(output_subfolder):
                os.makedirs(output_subfolder)

            output_path = os.path.join(output_subfolder, file_name)

            cv2.imwrite(output_path, wb_img)
            print(f"✅ Image corrigée et enregistrée : {output_path}")
            files_processed += 1

if files_processed == 0:
    print("⚠️ Aucune image n'a été traitée. Vérifie le dossier 'dataset'.")

print(" Balance des blancs appliquée avec succès !")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1245.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1370.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1319.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1351.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1241.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1340.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1230.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1327.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images/test/Bitter_Gourd/1357.jpg
✅ Image corrigée et enregistrée : enhanced_dataset/Vegetable Images

In [49]:
#data augumentation :

In [50]:
#augumentation sur toutes la data_set:
import cv2
import numpy as np
import os

# Fonction pour effectuer une rotation de l'image
def rotate_image(img, angle):
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, matrix, (w, h))
    return rotated

# Fonction pour ajuster la luminosité de l'image
def adjust_brightness(img, factor):
    return cv2.convertScaleAbs(img, alpha=factor, beta=0)

# Augmentation d'image et sauvegarde dans le dossier de sortie
input_folder = "enhanced_dataset"
output_folder = "augmented_dataset"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

files_processed = 0  # Compteur d'images traitées

for root, dirs, files in os.walk(input_folder):
    for file_name in files:
        img_path = os.path.join(root, file_name)

        if not file_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            continue

        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠️ Impossible de charger : {img_path}")
        else:
            # Augmentations
            rotated_img = rotate_image(img, 15)  # Rotation de 15 degrés
            bright_img = adjust_brightness(img, 1.2)  # Augmentation de la luminosité (facteur 1.2)

            # Sauvegarde des images augmentées
            relative_path = os.path.relpath(root, input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)

            if not os.path.exists(output_subfolder):
                os.makedirs(output_subfolder)

            # Enregistrer les images augmentées
            cv2.imwrite(os.path.join(output_subfolder, f"rotated_{file_name}"), rotated_img)
            cv2.imwrite(os.path.join(output_subfolder, f"bright_{file_name}"), bright_img)
            files_processed += 2  # 2 images générées par image traitée

if files_processed == 0:
    print("Aucune image n'a été augmentée.")
else:
    print("Augmentation d'images terminée !")

Augmentation d'images terminée !


In [51]:
import os

input_folder = "enhanced_dataset"
output_folder = "/content/augmented_dataset/Vegetable Images"

# Compter le nombre d'images avant augmentation
num_images_before = sum(len(files) for _, _, files in os.walk(input_folder))
print(f"Nombre d'images avant augmentation: {num_images_before}")

# Compter le nombre d'images après augmentation
num_images_after = sum(len(files) for _, _, files in os.walk(output_folder))
print(f"Nombre d'images après augmentation: {num_images_after}")

# Vérification
if num_images_after > num_images_before:
    print("Les images ont été augmentées avec succès !")
else:
    print("⚠️ Aucune augmentation détectée, vérifie le script !")


Nombre d'images avant augmentation: 21000
Nombre d'images après augmentation: 42000
Les images ont été augmentées avec succès !


In [52]:
!pip install tensorflow numpy matplotlib


In [60]:
#vérification de repartition de dataset

In [53]:
import os

dataset_path = "/content/augmented_dataset/Vegetable Images"  # Remplace par le chemin de ton dataset
subsets = ["test", "train", "validation"]

for subset in subsets:
    subset_path = os.path.join(dataset_path, subset)
    if os.path.exists(subset_path):
        total_images = sum([len(files) for _, _, files in os.walk(subset_path)])
        print(f"Nombre d'images dans {subset}: {total_images}")
    else:
        print(f"Le dossier {subset} n'existe pas !")


Nombre d'images dans test: 6000
Nombre d'images dans train: 30000
Nombre d'images dans validation: 6000


In [ ]:
train_count = sum([len(files) for _, _, files in os.walk(os.path.join(dataset_path, "train"))])
val_count = sum([len(files) for _, _, files in os.walk(os.path.join(dataset_path, "validation"))])
test_count = sum([len(files) for _, _, files in os.walk(os.path.join(dataset_path, "test"))])

total = train_count + val_count + test_count

train_percentage = (train_count / total) * 100
val_percentage = (val_count / total) * 100
test_percentage = (test_count / total) * 100

print(f"Train: {train_percentage:.2f}% ({train_count} images)")
print(f"Validation: {val_percentage:.2f}% ({val_count} images)")
print(f"Test: {test_percentage:.2f}% ({test_count} images)")


In [ ]:
#CHARGER data

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Définition des répertoires
train_dir = "/content/augmented_dataset/Vegetable Images/train"
val_dir = "/content/augmented_dataset/Vegetable Images/validation"
test_dir = "/content/augmented_dataset/Vegetable Images/test"

img_size = (224, 224)
batch_size = 32

# Générateur de données avec normalisation
datagen = ImageDataGenerator(rescale=1./255)

# Chargement des images
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

print("Données chargées avec succès !")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Charger ResNet50 sans la dernière couche
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Débloquer les 20 dernières couches pour le fine-tuning
for layer in base_model.layers[:-20]:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Ajouter des couches personnalisées
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.2)(x)  # Nouveau Dropout après le pooling global
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Déjà présent, bon choix
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)  # Augmenté à 0.4 pour plus de régularisation
x = Dense(15, activation='softmax')(x)  # 15 classes de légumes

# Créer et compiler le modèle
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()


In [ ]:
epochs = 8 # Nombre d'époques (peut être augmenté si besoin)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)


In [ ]:
import matplotlib.pyplot as plt

# Courbes de précision
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Courbe d’Accuracy')
plt.show()

# Courbes de perte
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Courbe de Loss')
plt.show()


In [ ]:
model.save('final_vegetable_model.keras')

In [ ]:
model.save('final_model.h5')

In [ ]:
from google.colab import files
files.download('/content/final_vegetable_model.h5')